In [125]:
import plotly.express as px
import plotly.graph_objects as go
import os
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import datetime
import numpy as np
from dataset.ipc2023_learning_domain_info import IPC2023_LEARNING_DOMAINS, DIFFICULTIES, PROBLEM_INDICES
from itertools import product
from IPython.display import display, HTML
from util.scrape_log import scrape_search_log
from pathlib import Path

In [126]:
_GENPLAN_PAPER_DIR=str(Path.home())+"/papers/manuscripts/genplan-23-kernels"
_ICAPS_PAPER_DIR=str(Path.home())+"/papers/manuscripts/icaps-24-kernels"

_GENPLAN_PLOT_DIR=f"{_GENPLAN_PAPER_DIR}/plots"
_ICAPS_PLOT_DIR=f"{_ICAPS_PAPER_DIR}/plots"

_PLOT_DIRS = [_GENPLAN_PLOT_DIR, _ICAPS_PLOT_DIR]
for p in _PLOT_DIRS:
    os.makedirs(p, exist_ok=True)

pd.options.display.float_format = "{:.1f}".format

In [127]:
## collect logs
# os.system("sh scripts_gnn/collect_cluster1_test.sh")
os.system("sh scripts_kernel/collect_gadi_test.sh")

collecting combined gpr logs...
  adding: icaps24_test_logs/childsnack_easy_p01_combined_gp.log (deflated 73%)
  adding: icaps24_test_logs/childsnack_easy_p02_combined_gp.log (deflated 73%)
  adding: icaps24_test_logs/childsnack_easy_p03_combined_gp.log (deflated 73%)
  adding: icaps24_test_logs/childsnack_easy_p04_combined_gp.log (deflated 73%)
  adding: icaps24_test_logs/childsnack_easy_p05_combined_gp.log (deflated 74%)
  adding: icaps24_test_logs/childsnack_easy_p06_combined_gp.log (deflated 74%)
  adding: icaps24_test_logs/childsnack_easy_p07_combined_gp.log (deflated 74%)
  adding: icaps24_test_logs/childsnack_easy_p08_combined_gp.log (deflated 74%)
  adding: icaps24_test_logs/childsnack_easy_p09_combined_gp.log (deflated 74%)
  adding: icaps24_test_logs/childsnack_easy_p10_combined_gp.log (deflated 74%)
  adding: icaps24_test_logs/childsnack_easy_p11_combined_gp.log (deflated 74%)
  adding: icaps24_test_logs/childsnack_easy_p12_combined_gp.log (deflated 74%)
  adding: icaps24_te

0

In [128]:
MUNINN = "muninn"
GOOSE_LLG_CPU = "goose-genplan23"
LAMA = "lama-first"

MQ_GPR = "combined_gp"
MQ_SVR = "combined"
LWL_2 = "2lwl_4_linear-svr"
SVR_1 = "1wl_4_linear-svr"
SVR_2 = "1wl_4_quadratic-svr_none"
SVR_3 = "1wl_4_cubic-svr_none"
SVR_inf = "1wl_4_rbf-svr_none"
GPR = "1wl_4_gp_none"

_MODELS = [
    "blind",
    "hff",
    LAMA,
    MUNINN,
    # GOOSE_LLG_CPU,
    #   "gL4max",
    #   "gL4mean",
    "gI4max",
    "gI4mean",
    MQ_GPR,
    MQ_SVR,
    GPR,
    SVR_1,
    SVR_2,
    SVR_3,
    SVR_inf,
    LWL_2,
]

WLS = {
    MQ_GPR,
    MQ_SVR,
    GPR,
    SVR_1,
    SVR_2,
    SVR_3,
    SVR_inf,
    LWL_2,
}
CLASSICAL = {
    "blind",
    "hff",
    LAMA,
}
GNNS = {
    MUNINN,
    GOOSE_LLG_CPU,
    "gL4max",
    "gL4mean",
    "gI4max",
    "gI4mean",
}

MQ_SKIPPED_DOMAINS = {
    "blocksworld",
    "sokoban",
}

PLOT_MAX_STD = 1e3
PLOT_MAX_EXPANDED = 1e8
PLOT_MAX_COST = 1e4

In [129]:
RENAME = {
    MUNINN: "Mun",
    "gI4max": "g_max",
    "gI4mean": "g_mean",
    LAMA: "lama",
    GOOSE_LLG_CPU: "gGenPlan23",
    GPR: "gpr",
    SVR_1: "1svr",
    SVR_2: "2svr",
    SVR_3: "3svr",
    SVR_inf: "∞svr",
    LWL_2: "2lwl",
    MQ_SVR: "mq_svr",
    MQ_GPR: "mq_gpr",
}

PLOT_RENAME = {
    "gI4max": "GOOSE(max)",
    "gI4mean": "GOOSE(mean)",
    "hff": r"hFF",
}


def model_type(model):
    if model in GNNS: return "gnn"
    if model in WLS: return "wl" 
    if model in CLASSICAL: return "classical" 
    raise ValueError(model)


def rename(model):
    if model in RENAME:
        return RENAME[model]
    return model


def plot_rename(model):
    if model in PLOT_RENAME:
        return PLOT_RENAME[model]
    return model

In [130]:
with open("ipc-2023-learning-reference-costs.pkl", 'rb') as file:
    REFERENCE_COSTS = {domain : [] for domain in IPC2023_LEARNING_DOMAINS}
    for (domain, diff_in_int, problem_index), reference_cost in pickle.load(file).items():
        REFERENCE_COSTS[domain].append(reference_cost)
## (domain) -> [reference_costs_1, ..., reference_costs_90]
# REFERENCE_COSTS

In [131]:
""" Preprocess data """
def get_data_tmp(domain):
    data = {}

    for model in _MODELS:
        d = {
            "solved": [],
            "expanded": [],
            "cost": [],
            "first_h": [],
            "std": [],
            "quality": [],
            "c_opt": [],
            "c_opt_min": [],
            "difficulty": [],
            "time": [],
            "tried": [],
        }

        for diff_i, difficulty in enumerate(DIFFICULTIES):
            for prob_i, pf_name in enumerate(PROBLEM_INDICES):
                pf = f"../benchmarks/ipc2023-learning-benchmarks/{domain}/testing/{difficulty}/{pf_name}.pddl"
                assert os.path.exists(pf)
                stats = None
                if model=="muninn":
                    stats = {
                        "first_h": -1,
                        "solved": 0,
                        "time": -1,
                        "cost": -1,
                        "expanded": -1,
                        "evaluated": -1,
                        "tried": 1,
                    }
                    log_file = f"logs/muninn/{domain}_{difficulty}_{pf_name}.log"
                    plan_file = f"logs/muninn/{domain}_{difficulty}_{pf_name}.plan"
                    cost = 0
                    if os.path.exists(plan_file):
                        with open(plan_file, "r") as f:
                            for line in f.readlines():
                                if len(line) > 0 and line[0] == "(":
                                    cost += 1
                    if cost > 0:
                        stats["solved"] = 1
                        stats["cost"] = cost
                    for line in open(log_file, 'r').readlines():
                        if "[1]" in line and stats['solved'] == 0:
                            toks = line.replace(",", "").replace("(", "").split()
                            stats["first_h"] = float(toks[4])
                        elif "Walltime Used:" in line:
                            time_object = datetime.datetime.strptime(line.split()[-1], "%H:%M:%S")
                            stats["time"] = time_object.hour * 3600 + time_object.minute * 60 + time_object.second
                elif model == "goose-genplan23":
                    log_file = f"icaps24_test_logs_old/{domain}_{difficulty}_{pf_name}_{model}.log"
                elif model[0] == "g":
                    # gI4max
                    rep = {"I": "ilg", "L": "llg"}[model[1]]
                    L = model[2]
                    aggr = model[3:]
                    log_file = f"icaps24_test_logs_old/{domain}_{difficulty}_{pf_name}_{domain}_{rep}_L{L}_H64_{aggr}_r0.log"
                elif model in WLS:
                    desc = None
                    desc_helper = model
                    if model in {MQ_SVR, MQ_GPR} and domain not in MQ_SKIPPED_DOMAINS:
                        desc = model
                    elif model == MQ_SVR and domain in MQ_SKIPPED_DOMAINS:
                        desc_helper = SVR_1
                    elif model == MQ_GPR and domain in MQ_SKIPPED_DOMAINS:
                        desc_helper = GPR
                    if desc is None:
                        desc = f"ilg_{desc_helper}_H"
                    log_file = f"icaps24_test_logs/{domain}_{difficulty}_{pf_name}_{desc}.log"
                else:
                    assert model in CLASSICAL
                    log_file = f"logs/ipc2023_{model}/{domain}_{difficulty}_{pf_name}_{model}.log"

                if stats is None:
                    stats = scrape_search_log(log_file)

                if stats["time"] >= 1800:
                    stats["solved"] = 0
                    stats["time"] = 1800

                c = stats["cost"] if stats["solved"] else -1
                c_opt = REFERENCE_COSTS[domain][diff_i * 30 + prob_i]
                quality = c_opt / c
                quality = max(quality, 0)
                quality = min(quality, 1)
                stats["quality"]=(quality)
                stats["c_opt"]=(c_opt)
                stats["c_opt_min"]=min(c_opt, stats["cost"])
                stats["difficulty"]=(difficulty)

                for k in d:
                    d[k].append(stats[k] if k in stats else -1)

        data[model] = d

    return data

data_domain_helper = {}
for domain in IPC2023_LEARNING_DOMAINS:
    data_domain_helper[domain] = get_data_tmp(domain)

def get_data(domain):
    return data_domain_helper[domain]

def get_whole_df():
    ret = {"domain": [], "model": [], "model_type": [], "difficulty": [], "problem": []}
    for domain in IPC2023_LEARNING_DOMAINS:
        data = get_data(domain)
        stat_keys = set(data[_MODELS[0]].keys())
        for k in stat_keys:
            if k in ret:
                continue
            ret[k] = []
        for model in _MODELS:
            for i in range(90):
                ret["domain"].append(domain)
                ret["model"].append(rename(model))
                ret["model_type"].append(model_type(model))
                ret["problem"].append(i+1)
                for k in stat_keys:
                    ret[k].append(data[model][k][i])
    return pd.DataFrame(data=ret)
WHOLE_DF = get_whole_df()
# WHOLE_DF

In [132]:
""" visualise prediction vs min(returned plan length, reference plan length)"""
model = GPR
df = WHOLE_DF
df = df[df["solved"] == True]
df = df[df["model"] == rename(model)]
fig = px.scatter(df, x="c_opt_min", y="first_h", color="domain", log_x=True, log_y=True)
max_metric= 1e4
diagonal = px.line(x=[1, max_metric], y=[1, max_metric], color_discrete_sequence=['gray'])
fig = go.Figure(diagonal.data + fig.data, layout=fig.layout)
fig.show()

## Summary of scores

In [133]:
def get_table_df(models=None, metric="quality"):
    d = {
        "domain": [],
    }
    if models is None:
        models = _MODELS
    for config in models:
        d[rename(config)] = []

    sum_scores = {config: 0 for config in models}

    for domain in IPC2023_LEARNING_DOMAINS:
        data = get_data(domain)
        d["domain"].append(domain)

        for config in models:
            score = sum(data[config][metric])
            d[rename(config)].append(score)
            sum_scores[config] += score

    d["domain"].append("sum")
    for config in models:
        d[rename(config)].append(sum_scores[config])

    return pd.DataFrame(d)

In [134]:
print("TODO: find the floortile goose can solve")

for metric in [
    "solved",
    "quality",
]:
    scores_df = get_table_df(metric=metric)
    scores = scores_df.to_numpy()
    scores = scores[:, 1:]
    display(scores_df)

TODO: find the floortile goose can solve


,domain,blind,hff,lama,Mun,g_max,g_mean,mq_gpr,mq_svr,gpr,1svr,2svr,3svr,∞svr,2lwl
0,blocksworld,8,28,61,40,49,58,48,49,48,49,23,17,21,22
1,childsnack,9,26,35,11,19,20,20,18,21,20,13,13,16,20
2,ferry,10,68,68,46,64,72,65,77,76,74,62,38,27,60
3,floortile,2,12,11,0,0,0,1,2,2,2,0,0,0,0
4,miconic,30,90,90,30,90,90,88,90,90,90,90,30,30,64
5,rovers,15,34,67,15,25,29,38,52,47,45,32,32,25,33
6,satellite,12,65,89,18,31,29,35,43,37,37,23,20,22,30
7,sokoban,27,36,40,26,32,33,38,37,38,37,33,35,28,30
8,spanner,30,30,30,32,30,33,64,72,40,30,30,30,30,50
9,transport,9,41,66,17,38,35,41,52,50,49,33,34,24,34


,domain,blind,hff,lama,Mun,g_max,g_mean,mq_gpr,mq_svr,gpr,1svr,2svr,3svr,∞svr,2lwl
0,blocksworld,8.0,14.1,39.8,40.0,45.6,52.7,43.5,44.4,43.5,44.4,20.2,14.2,17.8,11.7
1,childsnack,9.0,20.1,22.0,11.0,17.6,19.9,19.3,17.4,19.4,18.9,11.5,10.8,13.5,16.6
2,ferry,10.0,67.6,62.9,46.0,63.9,71.9,64.7,76.7,75.6,73.6,61.5,35.4,26.2,59.6
3,floortile,2.0,11.2,10.0,0.0,0.0,0.0,0.9,1.8,1.8,1.8,0.0,0.0,0.0,0.0
4,miconic,30.0,88.5,81.6,30.0,89.2,89.1,87.5,89.4,89.1,89.0,86.9,28.8,28.7,61.4
5,rovers,15.0,32.7,65.2,14.2,19.7,24.8,30.9,40.6,37.9,36.4,26.6,26.2,21.8,26.9
6,satellite,12.0,63.8,87.3,18.0,24.6,21.6,30.9,39.0,32.0,32.9,19.1,16.0,17.7,24.9
7,sokoban,27.0,26.3,29.8,24.3,28.3,30.0,34.8,33.1,34.8,33.1,29.7,31.7,24.7,27.6
8,spanner,30.0,30.0,30.0,32.0,30.0,33.0,60.1,68.4,37.1,27.6,28.0,28.0,28.0,49.9
9,transport,9.0,39.3,64.0,17.0,35.6,31.6,39.3,50.5,46.9,46.3,29.6,29.1,21.8,31.8


## Plots

In [135]:
def plotall(metric, models, plot_reference_cost=True, log_yscale=True, ylim=None):
    save_dir = f"px/{metric}"
    os.makedirs(save_dir, exist_ok=True)

    models = [rename(model) for model in models]

    df = WHOLE_DF
    df = df[df['model'].isin(models)]
    for domain in IPC2023_LEARNING_DOMAINS:
        df_domain = df[df['domain'] == domain]
        fig = px.scatter(df_domain, x="problem", y=metric, title=domain, color="model", log_y=log_yscale, range_y=ylim, range_x=(0, 90))
        
        if plot_reference_cost:
            # otherwise, will loop the line around
            df_line = df_domain[df_domain['model'] == models[0]]
            reference_line = px.line(df_line, x="problem", y="c_opt", color_discrete_sequence=['black'], log_y=log_yscale)
            fig = go.Figure(data=fig.data + reference_line.data, layout=fig.layout)

        # fig.show()
        fig.write_html(f"{save_dir}/{domain}.html")
        for plot_dir in _PLOT_DIRS:
            paper_save_dir = f"{plot_dir}/{metric}"
            os.makedirs(paper_save_dir, exist_ok=True)
            fig.write_image(f"{paper_save_dir}/{domain}.pdf")

In [136]:
def plot_versus(model_x, model_y, metric):
    if metric == "expanded":
        max_metric = PLOT_MAX_EXPANDED
        log_scale = True
    elif metric == "cost":
        max_metric = PLOT_MAX_COST
        log_scale = True
    else:
        raise NotImplementedError(metric)

    save_dir = f"px/versus_{metric}"
    os.makedirs(save_dir, exist_ok=True)

    d = {"domain": [], plot_rename(model_x): [], plot_rename(model_y): []}
    for domain in IPC2023_LEARNING_DOMAINS:
        data = get_data(domain)
        for i in range(90):
            for model in [model_x, model_y]:
                if i >= len(data[model]["solved"]) or not data[model]["solved"][i]:
                    val = max_metric
                else:
                    val = data[model][metric][i]
                d[plot_rename(model)].append(val)
            d["domain"].append(domain)
    df = pd.DataFrame(data=d)
    diagonal = px.line(x=[1, max_metric], y=[1, max_metric], color_discrete_sequence=["gray"], line_dash_sequence=['dash'])
    h = px.line(x=[max_metric, max_metric], y=[1, max_metric], color_discrete_sequence=["gray"], line_dash_sequence=['dash'])
    v = px.line(x=[1, max_metric], y=[max_metric, max_metric], color_discrete_sequence=["gray"], line_dash_sequence=['dash'])
    scatter = px.scatter(
        df,
        x=plot_rename(model_x),
        y=plot_rename(model_y),
        color="domain",
        symbol="domain",
        log_x=log_scale,
        log_y=log_scale,
        range_x=(log_scale, max_metric * 2),
        range_y=(log_scale, max_metric * 2),
    )
    fig = go.Figure(diagonal.data + h.data + v.data + scatter.data, layout=scatter.layout)
    # fig = go.Figure(diagonal.data + scatter.data, layout=scatter.layout)
    fig.write_html(f"{save_dir}/{model_x}_vs_{model_y}.html")

    # fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
    R = 300
    fig.update_layout(
        # autosize=False,
        width=R,  # You can adjust the width as needed
        height=R,  # You can adjust the height as needed
        showlegend=False,
        # aspectratio=dict(x=1, y=1)
    )
    fig.update_xaxes(nticks=5)
    fig.update_yaxes(nticks=5)
    # fig.update_traces(opacity=.75)
    fig.update_layout(template='simple_white')

    for plot_dir in _PLOT_DIRS:
        save_dir = f"{plot_dir}/versus_{metric}"
        os.makedirs(save_dir, exist_ok=True)
        fig.write_image(f"{save_dir}/{model_x}_vs_{model_y}.pdf")

### Versus

In [137]:
metrics = ["expanded", "cost"]
pairs = [
    ("gI4max", "gI4mean"),
    # ("1wl_1", "2lwl_1"),
    (MQ_SVR, SVR_1),
    (MQ_SVR, LAMA),
    (SVR_1, "gI4mean"),
    (SVR_1, "hff"),
    # ("1wl_1", "1wl_4"),
    # ("2lwl_1", "2lwl_4"),
    # (GOOSE_LLG_CPU, "gI4mean"),
]
for metric, (mx, my) in product(metrics, pairs):
    plot_versus(model_x=mx, model_y=my, metric=metric)

### Expanded 

In [138]:
plotall(
    "expanded",
    ylim=(1, PLOT_MAX_EXPANDED),
    models=[
        "blind",
        "hff",
        # GOOSE_LLG_CPU,
        "gI4max",
        "gI4mean",
        MQ_SVR,
        SVR_1,
        SVR_2,
        SVR_3,
        SVR_inf,
    ],
)

### Plan quality


In [139]:
plotall(
    "cost",
    ylim=(1, PLOT_MAX_COST),
    models=[
        "blind",
        "hff",
        "muninn",
        "goose-genplan23",
        "gI4max",
        "gI4mean",
        "1wl",
        SVR_1,
        SVR_2,
        SVR_3,
        SVR_inf,
    ],
)

### Cumulative runtime

In [151]:
fig = px.ecdf(
    WHOLE_DF,
    x="time",
    y="solved",
    color="model",
    line_dash="model_type",
    ecdfnorm=None,
    log_x=True,
    range_x=(1e-3, 2000),
)
fig.show()
fig.write_html("px/cumulative_runtime.html")

### Cumulative expanded

In [141]:
exp_df = WHOLE_DF
exp_df["expanded"] = exp_df["expanded"] * exp_df["solved"]
fig = px.ecdf(
    exp_df,
    x="expanded",
    y="solved",
    color="model",
    line_dash="model_type",
    ecdfnorm=None,
    log_x=True,
)
fig.show()
fig.write_html("px/cumulative_expanded.html")

### Bayesian s.t.d. vs other metrics

In [142]:
df = WHOLE_DF.copy()
df = df[df["model"] == rename(GPR)]
metric = "expanded"
df[metric] = np.where(df["solved"], df[metric], PLOT_MAX_EXPANDED)
fig = px.scatter(
    df, x="std", y=metric, color="domain", symbol="difficulty", log_x=True, log_y=True, range_x=(1e-6, PLOT_MAX_STD), range_y=(1, PLOT_MAX_EXPANDED), hover_data=["problem"]
)
fig.show()
fig.write_html("px/expanded_vs_std.html")

### Hit WL colours

In [143]:
ret = {"domain": [], "model": [], "problem": [], "ratio": [], "itr": []}

ITRS = 4
for model, domain in product(WLS, IPC2023_LEARNING_DOMAINS):
    for diff_i, difficulty in enumerate(DIFFICULTIES):
        for prob_i, pf_name in enumerate(PROBLEM_INDICES):
            pf = f"../benchmarks/ipc2023-learning-benchmarks/{domain}/testing/{difficulty}/{pf_name}.pddl"
            assert os.path.exists(pf)
            desc = model
            log_file = f"icaps24_test_logs/{domain}_{difficulty}_{pf_name}_ilg_{desc}_H.log"

            if not os.path.exists(log_file):
                continue

            tmp = {}

            for line in open(log_file, "r").readlines():

                if "seen/unseen colours in itr" in line:
                    line = line.replace(":", "")
                    toks = line.split()
                    itr = int(toks[-3])
                    seen = int(toks[-2])
                    unseen = int(toks[-1])
                    ratio = seen / (seen + unseen)
                    tmp[itr] = ratio
                
            if len(tmp) == ITRS:
                for itr, ratio in tmp.items():
                    ret["domain"].append(domain)
                    ret["model"].append(rename(model))
                    ret["problem"].append(30 * diff_i + prob_i)
                    ret["ratio"].append(ratio)
                    ret["itr"].append(itr+1)

RATIO_DF = pd.DataFrame(data=ret)

In [144]:
def plot_ratio(model):
    save_dir = f"px/ratio"
    os.makedirs(save_dir, exist_ok=True)
    model = rename(model)

    df = RATIO_DF.copy()
    df = df[df['model']==model]
    for domain in IPC2023_LEARNING_DOMAINS:
        df_domain = df[df['domain'] == domain]
        fig = px.scatter(df_domain, x="problem", y="ratio", title=f"{model}_{domain}", color="itr", range_y=(0, 1), range_x=(0, 90))
        
        # fig.show()
        fig.write_html(f"{save_dir}/{model}_{domain}.html")
        plt.clf()

In [145]:
plot_ratio(model=SVR_1)
plot_ratio(model=SVR_2)
plot_ratio(model=SVR_3)
plot_ratio(model=SVR_inf)

<Figure size 640x480 with 0 Axes>

## More refined data

In [146]:
def get_metrics_df(data, models=None):
  d = {
    "config": [],
    "quality": [],
    "solved": [],
    "tried": [],
    "e_quality": [],
    "e_solved": [],
    "e_tried": [],
    "m_quality": [],
    "m_solved": [],
    "m_tried": [],
    "h_quality": [],
    "h_solved": [],
    "h_tried": [],
  }
  if models is None:
    models = _MODELS
  for config in models:
    d["config"].append(config)
    d["quality"].append(sum(data[config]["quality"]))
    d["solved"].append(sum(data[config]["solved"]))
    d["tried"].append(len(data[config]["solved"]))
    for difficulty in DIFFICULTIES:
      quality = 0
      solved = 0
      tried = 0
      for i, prob_solved in enumerate(data[config]["solved"]):
        if data[config]["difficulty"][i] == difficulty:
          quality += data[config]["quality"][i]
          solved += prob_solved
          tried += data[config]["tried"][i]
      d[f"{difficulty[0]}_quality"].append(quality)
      d[f"{difficulty[0]}_solved"].append(solved)
      d[f"{difficulty[0]}_tried"].append(tried)
  return pd.DataFrame(d)

In [147]:
for domain in IPC2023_LEARNING_DOMAINS:
    print(domain)
    df = get_metrics_df(get_data(domain))
    display(df)

blocksworld


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,blind,8.0,8,90,8.0,8,30,0.0,0,30,0.0,0,30
1,hff,14.1,28,90,14.1,28,30,0.0,0,30,0.0,0,30
2,lama-first,39.8,61,90,20.8,30,30,17.1,27,30,1.9,4,30
3,muninn,40.0,40,90,30.0,30,30,10.0,10,30,0.0,0,30
4,gI4max,45.6,49,90,28.1,30,30,14.8,16,30,2.7,3,18
5,gI4mean,52.7,58,90,27.2,30,30,21.9,24,30,3.6,4,20
6,combined_gp,43.5,48,90,26.8,30,30,16.7,18,30,0.0,0,30
7,combined,44.4,49,90,26.7,30,30,17.8,19,30,0.0,0,30
8,1wl_4_gp_none,43.5,48,90,26.8,30,30,16.7,18,30,0.0,0,30
9,1wl_4_linear-svr,44.4,49,90,26.7,30,30,17.8,19,30,0.0,0,30


childsnack


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,blind,9.0,9,90,9.0,9,30,0.0,0,30,0.0,0,30
1,hff,20.1,26,90,20.1,26,30,0.0,0,30,0.0,0,30
2,lama-first,22.0,35,90,18.5,29,30,3.0,5,30,0.6,1,30
3,muninn,11.0,11,90,11.0,11,30,0.0,0,30,0.0,0,30
4,gI4max,17.6,19,90,17.6,19,30,0.0,0,3,0.0,0,0
5,gI4mean,19.9,20,90,19.9,20,28,0.0,0,0,0.0,0,0
6,combined_gp,19.3,20,90,19.3,20,20,0.0,0,0,0.0,0,0
7,combined,17.4,18,90,17.4,18,30,0.0,0,30,0.0,0,0
8,1wl_4_gp_none,19.4,21,90,19.4,21,30,0.0,0,30,0.0,0,0
9,1wl_4_linear-svr,18.9,20,90,18.9,20,30,0.0,0,30,0.0,0,0


ferry


...

,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,blind,27.0,27,90,27.0,27,30,0.0,0,30,0,0,30
1,hff,26.3,36,90,23.2,30,30,3.1,6,30,0,0,30
2,lama-first,29.8,40,90,22.4,30,30,7.4,10,30,0,0,30
3,muninn,24.3,26,90,24.3,26,30,0.0,0,30,0,0,30
4,gI4max,28.3,32,90,26.3,30,30,2.0,2,18,0,0,0
5,gI4mean,30.0,33,90,27.0,30,30,3.0,3,23,0,0,0
6,combined_gp,34.8,38,90,26.8,30,30,8.0,8,30,0,0,0
7,combined,33.1,37,90,26.1,30,30,7.0,7,30,0,0,0
8,1wl_4_gp_none,34.8,38,90,26.8,30,30,8.0,8,30,0,0,0
9,1wl_4_linear-svr,33.1,37,90,26.1,30,30,7.0,7,30,0,0,0


spanner


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,blind,30.0,30,90,30.0,30,30,0.0,0,30,0.0,0,30
1,hff,30.0,30,90,30.0,30,30,0.0,0,30,0.0,0,30
2,lama-first,30.0,30,90,30.0,30,30,0.0,0,30,0.0,0,30
3,muninn,32.0,32,90,30.0,30,30,2.0,2,30,0.0,0,30
4,gI4max,30.0,30,90,30.0,30,30,0.0,0,13,0.0,0,0
5,gI4mean,33.0,33,90,30.0,30,30,3.0,3,21,0.0,0,0
6,combined_gp,60.1,64,90,26.1,30,30,30.0,30,30,4.0,4,4
7,combined,68.4,72,90,26.4,30,30,30.0,30,30,12.0,12,30
8,1wl_4_gp_none,37.1,40,90,27.1,30,30,10.0,10,30,0.0,0,0
9,1wl_4_linear-svr,27.6,30,90,27.6,30,30,0.0,0,30,0.0,0,0


transport


,config,quality,solved,tried,e_quality,e_solved,e_tried,m_quality,m_solved,m_tried,h_quality,h_solved,h_tried
0,blind,9.0,9,90,9.0,9,30,0.0,0,30,0,0,30
1,hff,39.3,41,90,28.3,30,30,11.0,11,30,0,0,30
2,lama-first,64.0,66,90,28.0,30,30,30.0,30,30,6,6,30
3,muninn,17.0,17,90,17.0,17,30,0.0,0,30,0,0,30
4,gI4max,35.6,38,90,27.9,30,30,7.8,8,17,0,0,0
5,gI4mean,31.6,35,90,27.5,30,30,4.1,5,26,0,0,0
6,combined_gp,39.3,41,90,28.6,30,30,10.7,11,11,0,0,0
7,combined,50.5,52,90,29.0,30,30,21.5,22,30,0,0,0
8,1wl_4_gp_none,46.9,50,90,27.6,30,30,19.2,20,30,0,0,0
9,1wl_4_linear-svr,46.3,49,90,28.3,30,30,18.0,19,30,0,0,0


## To GenPlan23 latex table

In [148]:
GENPLAN_MODELS = {
    "blind": "blind",
    "hff": "hff",
    "muninn": "muninn",
    "gI4max": "goosemax",
    "gI4mean": "goosemean",
    "1wl": "wl",
}

In [149]:
""" coverage_tall.tex and quality_tall.tex """

coverage_i = [5, 8, 11, 2]
quality_i = [4, 7, 10, 1]

for metric, indices in [("coverage", coverage_i), ("quality", quality_i)]:
    nnn = 0
    genplan_models_list = list(GENPLAN_MODELS.keys())
    scores_df = get_table_df(models=genplan_models_list)
    scores = scores_df.to_numpy()
    scores = scores[:,1:]

    ret  = r"\begin{tabular}{" + "c " + " ".join(["p{\coveragewidth}" for _ in range(len(GENPLAN_MODELS))]) + r"} \toprule "
    # ret  = r"\begin{tabular}{" + "c " + " ".join(["c" for _ in range(len(GENPLAN_MODELS))]) + r"} \toprule Domain "
    ret += r" & & & &\multicolumn{3}{c}{GOOSE} \\ \cmidrule(lr){5-7} Domain "

    for model in GENPLAN_MODELS.values():
        ret += f"& \\header{{\{model.replace('-', '')}}} "
    ret += r"\\ \midrule" + '\n'

    all_solved = {
    "easy": np.zeros(len(GENPLAN_MODELS)).astype(float),
    "medium": np.zeros(len(GENPLAN_MODELS)).astype(float),
    "hard": np.zeros(len(GENPLAN_MODELS)).astype(float),
    "all": np.zeros(len(GENPLAN_MODELS)).astype(float),
    }
    all_quality = {
    "easy": np.zeros(len(GENPLAN_MODELS)),
    "medium": np.zeros(len(GENPLAN_MODELS)),
    "hard": np.zeros(len(GENPLAN_MODELS)),
    "all": np.zeros(len(GENPLAN_MODELS)),
    }

    for domain in IPC2023_LEARNING_DOMAINS:
        ret += f"\\multirow{{4}}{{*}}{{{domain}}}"
        data = (get_metrics_df(get_data(domain), models = genplan_models_list).to_numpy())

        # coverage then cost
        solved = {
            "easy": data[:,indices[0]].T,
            "medium": data[:,indices[1]].T,
            "hard": data[:,indices[2]].T,
            "all": data[:,indices[3]].T,
        }

        for difficulty in ["easy", "medium", "hard", "all"]:
            scores_domain = solved[difficulty]
            sorted_scores_domain = sorted(scores_domain, reverse=True)
            all_solved[difficulty] += np.array(scores_domain).astype(float)
            for i in range(len(solved["easy"])):
                chunks = []
                best_score = max(scores_domain)
                score = solved[difficulty][i]
                if metric == "quality":
                    display_score = f"{score:.1f}"
                else:
                    display_score = score
                if score == 0:
                    chunks.append("-")
                elif score==sorted_scores_domain[0]:
                    chunks.append(f"\\first{{{display_score}}}")
                elif score==sorted_scores_domain[1]:
                    chunks.append(f"\\second{{{display_score}}}")
                elif score==sorted_scores_domain[2]:
                    chunks.append(f"\\third{{{display_score}}}")
                else:
                    chunks.append(f"{display_score}")

                chunk = " & ".join(chunks)
                ret += f" & {chunk}"

            if difficulty != "all":
                ret += "\\\\ \n"

        if domain==IPC2023_LEARNING_DOMAINS[-1]:
            ret += "\\\\ \midrule \n"
        else:
            ret += "\\\\ \cmidrule(lr){2-" + str(len(GENPLAN_MODELS)+1) + "} \n"

    ret += f"\\multirow{{4}}{{*}}{{sum}}"

    for difficulty in ["easy", "medium", "hard", "all"]:
        for i in range(len(all_solved["easy"])):
            chunks = []

            scores_domain = all_solved[difficulty]

            sorted_scores_domain = sorted(scores_domain, reverse=True)
            sorted_scores_domain = [round(x, 1) for x in sorted_scores_domain]
            best_score = max(scores_domain)
            score = round(all_solved[difficulty][i], 1)
            if metric == "quality":
                display_score = f"{score:.1f}"
            else:
                display_score = f"{int(score):d}"
            if score == 0:
                chunks.append("-")
            elif score==sorted_scores_domain[0]:
                chunks.append(f"\\first{{{display_score}}}")
            elif score==sorted_scores_domain[1]:
                chunks.append(f"\\second{{{display_score}}}")
            elif score==sorted_scores_domain[2]:
                chunks.append(f"\\third{{{display_score}}}")
            else:
                chunks.append(f"{display_score}")

            chunk = " & ".join(chunks)
            ret += f" & {chunk}"

        if difficulty != "all":
            ret += "\\\\ \n"

    ret += r" \\ \bottomrule\end{tabular}"

    f = open(f"{_GENPLAN_PAPER_DIR}/figures/{metric}_tall.tex", 'w')
    f.write(ret)
    f.close()

KeyError: '1wl'

In [ ]:
""" condensed versions """

coverage_i = [5, 8, 11, 2]
quality_i = [4, 7, 10, 1]

for metric, indices in [("coverage", coverage_i), ("quality", quality_i)]:
    nnn = 0
    genplan_models_list = list(GENPLAN_MODELS.keys())
    scores_df = get_table_df(models=genplan_models_list)
    scores = scores_df.to_numpy()
    scores = scores[:,1:]

    ret  = r"\begin{tabular}{" + "c " + " ".join(["p{\coveragewidth}" for _ in range(len(GENPLAN_MODELS))]) + r"} \toprule "
    # ret  = r"\begin{tabular}{" + "c " + " ".join(["c" for _ in range(len(GENPLAN_MODELS))]) + r"} \toprule Domain "
    ret += r" & & & &\multicolumn{3}{c}{GOOSE} \\ \cmidrule(lr){5-7} Domain "

    for model in GENPLAN_MODELS.values():
        ret += f"& \\header{{\{model.replace('-', '')}}} "
    ret += r"\\ \midrule" + '\n'

    all_solved = {
    "easy": np.zeros(len(GENPLAN_MODELS)).astype(float),
    "medium": np.zeros(len(GENPLAN_MODELS)).astype(float),
    "hard": np.zeros(len(GENPLAN_MODELS)).astype(float),
    "all": np.zeros(len(GENPLAN_MODELS)).astype(float),
    }
    all_quality = {
    "easy": np.zeros(len(GENPLAN_MODELS)),
    "medium": np.zeros(len(GENPLAN_MODELS)),
    "hard": np.zeros(len(GENPLAN_MODELS)),
    "all": np.zeros(len(GENPLAN_MODELS)),
    }

    for domain in IPC2023_LEARNING_DOMAINS:
        ret += f"{{{domain}}}"
        data = (get_metrics_df(get_data(domain), models = genplan_models_list).to_numpy())

        # coverage then cost
        solved = {
            "easy": data[:,indices[0]].T,
            "medium": data[:,indices[1]].T,
            "hard": data[:,indices[2]].T,
            "all": data[:,indices[3]].T,
        }

        for difficulty in ["easy", "medium", "hard", "all"]:
            scores_domain = solved[difficulty]
            sorted_scores_domain = sorted(scores_domain, reverse=True)
            all_solved[difficulty] += np.array(scores_domain).astype(float)
            for i in range(len(solved["easy"])):
                chunks = []
                best_score = max(scores_domain)
                score = solved[difficulty][i]
                if metric == "quality":
                    display_score = f"{score:.1f}"
                else:
                    display_score = score
                if score == 0:
                    chunks.append("-")
                elif score==sorted_scores_domain[0]:
                    chunks.append(f"\\first{{{display_score}}}")
                elif score==sorted_scores_domain[1]:
                    chunks.append(f"\\second{{{display_score}}}")
                elif score==sorted_scores_domain[2]:
                    chunks.append(f"\\third{{{display_score}}}")
                else:
                    chunks.append(f"{display_score}")

                chunk = " & ".join(chunks)
                if difficulty in {"easy", "medium", "hard"}:
                    continue
                ret += f" & {chunk}"

            # if difficulty != "all":
            #     ret += "\\\\ \n"

        if domain==IPC2023_LEARNING_DOMAINS[-1]:
            ret += "\\\\ \midrule \n"
        else:
            ret += "\\\\ \n"

    ret += f"{{sum}}"

    for difficulty in ["all"]:
        for i in range(len(all_solved["easy"])):
            chunks = []

            scores_domain = all_solved[difficulty]

            sorted_scores_domain = sorted(scores_domain, reverse=True)
            sorted_scores_domain = [round(x, 1) for x in sorted_scores_domain]
            best_score = max(scores_domain)
            score = round(all_solved[difficulty][i], 1)
            if metric == "quality":
                display_score = f"{score:.1f}"
            else:
                display_score = f"{int(score):d}"
            if score == 0:
                chunks.append("-")
            elif score==sorted_scores_domain[0]:
                chunks.append(f"\\first{{{display_score}}}")
            elif score==sorted_scores_domain[1]:
                chunks.append(f"\\second{{{display_score}}}")
            elif score==sorted_scores_domain[2]:
                chunks.append(f"\\third{{{display_score}}}")
            else:
                chunks.append(f"{display_score}")

            chunk = " & ".join(chunks)
            ret += f" & {chunk}"

        if difficulty != "all":
            ret += "\\\\ \n"

    ret += r" \\ \bottomrule\end{tabular}"

    f = open(f"{_GENPLAN_PAPER_DIR}/figures/small_{metric}_tall.tex", 'w')
    f.write(ret)
    f.close()

# To ICAPS24 latex table

In [ ]:
""" coverage_tall.tex """
nnn = 0

scores_df = get_table_df()
scores = scores_df.to_numpy()
scores = scores[:,1:]

# ret  = r"\begin{tabular}{ c " + " ".join(["c" for _ in range(4*len(_MODELS))]) + r"} \toprule Domain "
ret  = r"\begin{tabular}{" + "c " + " ".join(["p{\coveragewidth}" for _ in range(len(_MODELS))]) + r"} \toprule Domain "

for model in _MODELS:
    ret += f"& \\header{{\{model.replace('-', '')}}} "
ret += r"\\ \midrule" + '\n'

all_solved = {
  "easy": np.zeros(len(_MODELS)).astype(int),
  "medium": np.zeros(len(_MODELS)).astype(int),
  "hard": np.zeros(len(_MODELS)).astype(int),
  "all": np.zeros(len(_MODELS)).astype(int),
}
all_quality = {
  "easy": np.zeros(len(_MODELS)),
  "medium": np.zeros(len(_MODELS)),
  "hard": np.zeros(len(_MODELS)),
  "all": np.zeros(len(_MODELS)),
}

for domain in IPC2023_LEARNING_DOMAINS:
    ret += f"\\multirow{{4}}{{*}}{{{domain}}}"
    data = (get_metrics_df(get_data(domain)).to_numpy())

    # coverage then cost
    solved = {
        "easy": data[:,5].T,
        "medium": data[:,8].T,
        "hard": data[:,11].T,
        "all": data[:,2].T,
    }

    for difficulty in ["easy", "medium", "hard", "all"]:
        scores_domain = solved[difficulty]
        sorted_scores_domain = sorted(scores_domain, reverse=True)
        all_solved[difficulty] += np.array(scores_domain).astype(int)
        for i in range(len(solved["easy"])):
            chunks = []
            best_score = max(scores_domain)
            score = solved[difficulty][i]
            if score == 0:
                chunks.append("-")
            elif score==sorted_scores_domain[0]:
                chunks.append(f"\\first{{{score}}}")
            elif score==sorted_scores_domain[1]:
                chunks.append(f"\\second{{{score}}}")
            elif score==sorted_scores_domain[2]:
                chunks.append(f"\\third{{{score}}}")
            else:
                chunks.append(f"{score}")

            chunk = " & ".join(chunks)
            ret += f" & {chunk}"

        if difficulty != "all":
            ret += "\\\\ \n"

    if domain==IPC2023_LEARNING_DOMAINS[-1]:
        ret += "\\\\ \midrule \n"
    else:
        ret += "\\\\ \cmidrule(lr){2-" + str(len(_MODELS)+1) + "} \n"

ret += f"\\multirow{{4}}{{*}}{{sum}}"

for difficulty in ["easy", "medium", "hard", "all"]:
    for i in range(len(all_solved["easy"])):
        chunks = []

        scores_domain = all_solved[difficulty]

        sorted_scores_domain = sorted(scores_domain, reverse=True)
        sorted_scores_domain = [round(x, 1) for x in sorted_scores_domain]
        best_score = max(scores_domain)
        score = all_solved[difficulty][i]
        if score == 0:
            chunks.append("-")
        elif score==sorted_scores_domain[0]:
            chunks.append(f"\\first{{{score}}}")
        elif score==sorted_scores_domain[1]:
            chunks.append(f"\\second{{{score}}}")
        elif score==sorted_scores_domain[2]:
            chunks.append(f"\\third{{{score}}}")
        else:
            chunks.append(f"{score}")

        chunk = " & ".join(chunks)
        ret += f" & {chunk}"

    if difficulty != "all":
        ret += "\\\\ \n"

ret += r" \\ \bottomrule\end{tabular}"

f = open(f"{_ICAPS_PAPER_DIR}/figures/coverage_tall.tex", 'w')
f.write(ret)
f.close()